# Overall Term Frequencies

With the TEDtalks-all dataset created, we have 1747 talks with which to work. This is a small corpus, and so the usual reasons for shrinking the feature set for the texts do not apply, but as we begin our survey of the contents of the TED talks we wanted to be mindful of standards that had emerged both so that our results would be comparable to the work of others but also so that we could potentially scale up the work here without having to re-think the foundations.

## Summary

In this notebook we load the complete dataset of both TED and TED+ talks. (No TEDx talks.) We use Python's `Sci-Kit Learn` library to create a document - term frequency matrix with a shape of 1747 x 50379. Summing the words to get a total for each word across all talks in the dataset, we then hand-inspect the totals and discover that there is a small list of numbers that recur. We approach mapping those numbers in two trials before returning to the task of getting a clean frequency list with no repetitions or other oddities.

## Imports and Data

In [1]:
# Imports
import pandas as pd, re, csv, nltk
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Load the Data
df = pd.read_csv('../output/TEDall_speakers.csv')
df.shape

(1747, 27)

**To Do**: Edit the CSV to remove the vestigial index column at the start of each line. Then use `df.set_index('Talk_ID')`.

## Frequencies

The sole purpose of this notebook is to establish how we are going to elicit our features, our words, from the collection of talks. Thus, the only column we are interested in is the one with the texts of the talks. As we move forward, however, we will want to decide if we are simply going to append ~30,000 columns to a version of the extant CSV or create a separate CSV for each experiment. 

For this first experiment, we will keep it simple, creating two lists, one of the URLs and one of the texts. The URLs are unique, human-friendly identifiers for the talks. (We can, perhaps, make them a bit more friendly by modifying them a bit, subtracting `https://www.ted.com/talks/` from each.)

In [3]:
urls  = df.public_url.tolist()
texts = df.text.tolist()

There are a number of ways to get term frequencies, but **SciKit-Learn**'s `CountVectorizer` is, I think, the way to go, since it will work well with the other vectorizers and models also available in `sklearn`.

In our first experiment, we run `CountVectorizer` unadorned. The default options are: lowercase everything, get rid of all punctuation, make a word out of anything more than two characters long. The only thing that might not be welcome is the splitting of contractions. For now, we will leave things as they are. (Also, please note, no stopwords were used, so we have an unfiltered word list.)

For this current work, we are running `fit()` and `transform()` separately, but since `fit()` just calculates the parameters and saves them as an internal objects state `transform()`  applies the transformation to a particular set of examples (the ones we just fitted), the two operations are usually simply done at the same time as `fit_transform()`.

In [6]:
# If you want to pass options, pass them here:
vec = CountVectorizer()

# fit the model to the data 
vecs = vec.fit(texts)

# transform the data according to the fitted model
X = vecs.transform(texts)

# see how many features we have
X.shape

(1747, 50379)

This is our baseline.

## Frequency Totals per Word

We can total up our columns for each feature (word), which is something we will be doing per year, per gender, per discipline. Here, we take the vector describing a word and sum it. We then pair the sum with the word in a tuple, which we then sort by frequency. 

(I'm doing it this way because it appears to be the way to do it, but it also strikes me that there should be a way to do this within the array itself, or, perhaps, to do it through **pandas**.)

We save the results to a CSV file so that we can hand-check the words: are these the results we expected? (We don't want any weirdness affecting our overall results.) The hand inspection looks good. I didn't see anything in words 4 or above in frequency that looked off. (So, the simplest solution works!) What I did note was the frequency of certain **numbers**: **100**, **12**, etc. This might be worth taking a closer look: are there *power numbers*? (I am thinking here of Alan Dundes' essay on the "power" of three in American culture.)

**To Do**: It would be nice to be able grab all words of a certain frequency, or range of frequencies.

---
**Follow-up**: whenever I attempt some version of
```python
for item in vecs:
    if vecs.vocabulary_.get(item) == 1691:
        print(item)
```
I get **`TypeError: 'CountVectorizer' object is not iterable`**. My best guess, for now, is that we need to use the tuple above to get this information.

---


In [ ]:
# summing up the counts for each word
sum_words = bow.sum(axis=0)

# create a tuple
words_freq = [(word, sum_words[0, idx]) for word, idx in vecs.vocabulary_.items()]

# sort the tuple
words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)

# check the results of our work by printing the top 20 more frequent words
print(words_freq[0:20])

In [ ]:
# with open('../output/word_freq.csv','w') as out:
#     csv_out = csv.writer(out)
#     csv_out.writerow(['word','count'])
#     for row in words_freq:
#         csv_out.writerow(row)

A hand inspection of the output above turned up a couple of interesting issues: that there are parentheticals mixed in with the text of the talks (see 01-Terms-02-Parentheticals) and numbers feature in the talks regularly (see 01-Terms-03-Numbers). 

There are also some interesting tokenization dimensions to a crowd-sourced transcription technology:

| Term    | Freq |
|:---------|---:|
|aa       |12 |
|aaa      |7  |
|aaaa     |2  |
|aaaaa    |1  |
|aaaaaaaah|1  |
|aaaaaaah |1  |
|aaaaaah  |3  |
|aaaah    |2  |
|aaaahhh  |1  |
|aaah     |3  |
|aag      |1  |
|aah      |10 |

There are an equal number of alternate spellings for *shh*. There's also this oddity:

| Term    | Freq |
|:---------|---:|
|FALSE    |115|
|TRUE     |909|

It appears to be simply the occurrences of *true* and *false* but somehow the vectorizer thinks they are Booleans?

In [26]:
onetext = nltk.Text(re.sub("[^a-zA-Z0-9'()]"," ",'\n'.join(texts)).split())

In [55]:
onetext.concordance("Marimba")

Displaying 5 of 5 matches:
nd perhaps even lies underneath the marimba and you play something that's so in
thing like this let me change (Soft marimba sounds) (Soft marimba sounds end) S
 change (Soft marimba sounds) (Soft marimba sounds end) Something that's so unb
ls this dynamic may well work (Soft marimba sounds) (Soft marimba sounds end) I
ll work (Soft marimba sounds) (Soft marimba sounds end) It may be that in other


The rest of the hand-examination turns up no other issues, so the basic vectorization built into `sklearn` appears to be satisfactory, with the only exception being its breaking of contractions at the apostrophe: e.g., *isn't* becomes *isn* and *'t*, with the latter being thrown away as too small -- which means that the indefinite article *a* is also not present in the frequencies. 

## Revised Frequencies without Parentheticals

For more on the work leading up to this regex, see the `01-Terms-02-Parentheticals` notebook: it contains a number of experiments, which were vexed by sklearn's vectorizer only accepting strings as inputs. (While gensim appears to accept texts as lists of words, we are hoping, I think, to keep our code base as simple as possible.) The eventual workaround is to clean the texts of the top 20 parentheticals, joining the list back to a string, and then pushing the result to sklearn. This is less than optimum, but the hack works and appears to be a hack performed elsewhere by others.

The number "2" below in the variables reveals that this was the second experiment (of three). If we could figure out a way to handle removing the items in the list below at the level of the string, we could pass that function to `CounterVectorizer()` as a `preprocessor = that_function`. 

For now, the first thing we have is the list of the top 20 parentheticals, some of which are two words:

In [9]:
stopped_parens = [ "(Laughter)", "(Applause)", "(Music)", "(Video)", "(Laughs)", 
                  "(applause ends)", "(Audio)", "(Singing)", "(music ends)", "(Cheers)", 
                  "(Cheering)", "(Recording)", "(Beatboxing)", "(Audience)", 
                  "(guitar strum)", "(clicks metronome)", "(Sighs)", "(Guitar)", 
                  "(marimba sounds)", "(drum sounds)" ]
len(stopped_parens)

20

The function below roundtrips the string:

**TO DO**: I just realized that this does not work on our two-word texts above.

In [10]:
def paren_cleanse (text):
    result = ' '.join([ word for word in re.sub("[^a-zA-Z0-9'()]"," ", text).lower().split() 
              if word not in stopped_parens ])
    return result

clean_texts = [ paren_cleanse(text) for text in texts]

# Sample our results
print(clean_texts[0][0:200])

thank you so much chris and it's truly a great honor to have the opportunity to come to this stage twice i'm extremely grateful i have been blown away by this conference and i want to thank all of you


In [11]:
vec2 = CountVectorizer(preprocessor = paren_cleanse)
X2 = vec2.fit_transform(texts)
X2.shape

(1747, 50301)

In [12]:
df_base = pd.DataFrame(X.toarray(), columns = vecs.get_feature_names())
df2 = pd.DataFrame(X2.toarray(), columns = vec2.get_feature_names())

In [13]:
term = 'beatboxing'
print(df_base[term].sum(), df2[term].sum())

23 5


In [14]:
def clean_text(rgx_list, text):
    new_text = text
    for rgx_match in rgx_list:
        new_text = re.sub(rgx_match, ' ', new_text, flags=re.IGNORECASE)
    return new_text

print(clean_text(stopped_parens, texts[235]))

  Let's just get started here.    Okay, just a moment.    (Whirring)    All right. ( ) Oh, sorry.    ( ) ( )    Thank you.    ( )  


This code is not working: it should be pulling out the parentheses as well.

In [15]:
reclean = [ clean_text(stopped_parens, text) for text in texts ]

vec3 = CountVectorizer()
X3 = vec3.fit_transform(reclean)
X3.shape

df3 = pd.DataFrame(X3.toarray(), columns = vec3.get_feature_names())

'music' in vec3.get_feature_names()

In [ ]:
test = "Laughter is the best medicine. (Laughter.) But 

In [ ]:
# term = 'laughter'
# print(df_base[term].sum(), df3[term].sum())